In [62]:
from transformers import pipeline
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

#print(torch.cuda.device_count())
#os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

#モデル読み込み
model_name = "meta-llama/Llama-2-7b-chat-hf"

model_name = "tokyotech-llm/Swallow-MS-7b-v0.1"
model_name = "cyberagent/open-calm-7b"

model_name="Local-Novel-LLM-project/Vecteus-v1"
#model_name="nvidia/Llama3-ChatQA-1.5-8B"

model_name="../model/------llm-models-hf-2ja1_part_inst_all_lr_1e-5"
model_name="../model/------llm-models-hf-2ja1_part_inst_all_lr_1e-5"
model_name="../X_merge/merged_models/0507test"
model_name="../../llm/models/hf/step41300",
model_name="../model/------llm-models-hf-step41300_inst_all-parquet_lr_1e-4"
model_name="../model/------llm-models-hf-step41300_inst_all-parquet_lr_1e-4"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)


In [63]:

tokens=[5092, 272, 1045, 2850, 327, 8,11267, 269, 564, 298, 2]
tokenizer.decode(tokens,)

'### 応答:\n はい、可能です</s>'

In [64]:
t_list=[
"\n\n### 応答:\n",
"\n\n### 応答:\n「",
"\n\n### 応答:\nああ",
"\n\n### 応答:\naaa",
"1\n\n### 応答:\naaa",
"あ\n\n### 応答:\naaa",
"」\n\n### 応答:\naaa",
]
t_list=[
    "",
    "a \n a",
"a\n   ",
"\n\n### 指示:\n",
"\n\n### 指示:\n「",
"\n\n### 指示:\nああ",
"\n\n### 指示:\naaa",
"1\n\n### 指示:\naaa",
"あ\n\n### 指示:\naaa",
"」\n\n### 指示:\naaa",
]
for t in t_list:
    print(tokenizer.encode(t,add_special_tokens=False),tokenizer.tokenize(t))


[] []
[355, 8, 355] ['▁a', '\n', '▁a']
[355, 8] ['▁a', '\n']
[8, 8, 5092, 272, 3994, 327, 8] ['\n', '\n', '▁###', '▁', '指示', ':', '\n']
[8, 8, 5092, 272, 3994, 327, 8, 337] ['\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁「']
[8, 8, 5092, 272, 3994, 327, 8, 8722] ['\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁ああ']
[8, 8, 5092, 272, 3994, 327, 8, 272, 55841] ['\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁', 'aaa']
[272, 279, 8, 8, 5092, 272, 3994, 327, 8, 272, 55841] ['▁', '1', '\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁', 'aaa']
[4543, 8, 8, 5092, 272, 3994, 327, 8, 272, 55841] ['▁あ', '\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁', 'aaa']
[272, 285, 8, 8, 5092, 272, 3994, 327, 8, 272, 55841] ['▁', '」', '\n', '\n', '▁###', '▁', '指示', ':', '\n', '▁', 'aaa']


In [65]:
tokenizer.encode(t)

[272, 285, 8, 8, 5092, 272, 3994, 327, 8, 272, 55841, 2]

In [66]:
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )


Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.61s/it]


In [67]:
#model.load_adapter(model_path)

In [68]:
#層の表示
for name, param in model.named_parameters():
    print(name)
    if "gate" in name:
        print("***",name)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
*** model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
*** model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
*** mode

In [69]:
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()



In [70]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=50, 
              repetition_penalty=1.2,
              temperature=0.6,
              #repetition_penalty=1.,
              #temperature=0.1,
              #top_p=1.0,
              #top_k=0.
              )


In [71]:
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n\n### 応答:\n"

def gen_prompt(q):
    return f"{question_template}{q}{answer_template}"
    return question_template+q+answer_template

gen_prompt("こんにちは")

'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\nこんにちは\n\n### 応答:\n'

In [72]:
questions=[
"こんにちは",
"元気ですか?",
"いい天気ですね",
"明日の天気は?",
"計算してください 1+1はいくつですか｡",
"フィボナッチ数列を生成するpythonのコードを作成してください",
"ドラえもんの友達はだれですか?",
"レイリー散乱とはなんですか",
"hello!",
"今の天気は晴れです｡この結果をjsonで出力してください｡ ",
"純粋理性批判はたぬきに理解できますか?",
]
#questions=[
#    "アスキーアートを出力して",
#]

for question in questions:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    res=pipe(inp)[0]["generated_text"][len(inp):]
    print(res)
    print(tokenizer.encode(res)[:4])

--------------------
こんにちは


/home/hatakeyama/miniconda3/envs/llmeval2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 「こんにちは」という言葉には、「あなたがここに来て下さってありがとうございます」「私もあなたのお役に立ちたいです」といった意味があります。「こんにちは」と言われた方は「自分のことを認めてくれて嬉しいな」とか「この人のために何かしたい
[337, 3124, 285, 319]
--------------------
元気ですか?
 はい!元気ですよ～! 最近、私がハマっているものがあります。それは「お菓子作り」です。今まで作ったことがなかったのですが、「自分で作ったお菓子なら美味しいはず!」という謎の自信から挑戦
[11267, 309, 1785, 1509]
--------------------
いい天気ですね
 はい!今日も良い天気で気持ちがいいですね。 最近では、晴れた日でも雨具や防寒着など必須アイテムがありますね。 これらのアイテムは、登山者にとって重要な役
[11267, 309, 620, 281]
--------------------
明日の天気は?
 天候に関しては、明日も晴れる可能性が高いようです。ただし、雨雲や雷雲の発生状況によっては、一時的な降雨がある場合もあります。また、風向きなどにも注意してくださいね!
[10250, 3598, 953, 304]
--------------------
計算してください 1+1はいくつですか｡
 はい,答えは「2」になります.これは,数学的な考え方から導き出されますが,この場合も同じように考えることができます.例えば,次のような問題があります:「AさんがBさんに50ドル渡しました
[11267, 287, 1302, 276]
--------------------
フィボナッチ数列を生成するpythonのコードを作成してください
 1. 0,2,3,...,n-1がそれぞれ異なる値であることを確認します(この場合、n=5)。次のように計算できます: n = (n - 1)/2 + 
[272, 279, 284, 272]
--------------------
ドラえもんの友達はだれですか?
 「ドラえもん」には様々なキャラクターが登場します。「ドラえもん」「ジャイアン」「スネ夫」「しずかちゃん」「出木杉くん」「大山先生(大山のぶ代)」などがありますね。これらのキャラクターたちは

KeyboardInterrupt: 

In [48]:
#tokenizer.piece_size = 100000
#tokenizer.add_special_tokens({'additional_special_tokens': ['\\n']})
#tokenizer.add_prefix_space=False

res=pipe(inp)[0]["generated_text"][:]
#tokenizer.encode("### 応答:\n はい、可能です")
res

'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n純粋理性批判はたぬきに理解できますか?\n\n### 応答:\n はい、可能です!純粋理性批判(1789年)では、人間が持つ「知」や「意識」、「感情」「欲望」などについて考察されています。「知」に関して言'

In [36]:
tokenizer.decode([11267]),res[0],tokenizer.encode("はい")

('はい', ' ', [11267, 2])

# ベンチマーク問題での応答評価

In [ ]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=50, 
              repetition_penalty=1.,
              temperature=0.1,
              top_p=1.0,
              top_k=0.
              )

In [ ]:
#ベンチマーク問題への応答性の軽い評価
import pandas as pd
jaster_path="bench_data/jaster.txt"
with open(jaster_path,"r") as f:
    input_list=f.readlines()

for question in input_list:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    print("出力")
    print(pipe(inp)[0]["generated_text"][len(inp):])

In [ ]:
jmt_path="bench_data/jmt.csv"
df=pd.read_csv(jmt_path)
input_list=df["input"].to_list()
for question in input_list:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    print("出力")
    print(pipe(inp)[0]["generated_text"][len(inp):])